In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf
from datetime import datetime, timedelta

def calculate_portfolio_metrics(weights, returns):
    """Calculate portfolio return and risk metrics"""
    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_vol = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe = (portfolio_return - 0.02) / portfolio_vol  # Assuming risk-free rate = 2%
    return -sharpe  # Negative because we want to maximize

def optimize_portfolio(returns):
    """Optimize portfolio weights to maximize Sharpe ratio"""
    n_assets = returns.shape[1]
    
    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # weights sum to 1
    ]
    
    # Bounds (0 to 1 for each weight)
    bounds = tuple((0, 1) for _ in range(n_assets))
    
    # Initial guess (equal weights)
    init_weights = np.array([1/n_assets] * n_assets)
    
    # Optimize
    result = minimize(calculate_portfolio_metrics, 
                     init_weights,
                     args=(returns,),
                     method='SLSQP',
                     bounds=bounds,
                     constraints=constraints)
    
    return result.x if result.success else init_weights

def rolling_optimization(price_data, window_size=60):
    """Perform rolling window optimization"""
    returns = price_data.pct_change()
    optimal_weights = pd.DataFrame(index=returns.index, columns=returns.columns)
    
    for i in range(window_size, len(returns)):
        # Get window of returns
        window_returns = returns.iloc[i-window_size:i]
        
        # Optimize for this window
        weights = optimize_portfolio(window_returns)
        
        # Store the weights
        optimal_weights.iloc[i] = weights
    
    # Forward fill the weights for any missing periods
    optimal_weights = optimal_weights.fillna(method='ffill')
    
    return optimal_weights

def calculate_optimized_portfolio_performance(price_data, optimal_weights):
    """Calculate portfolio performance using optimized weights"""
    returns = price_data.pct_change()
    portfolio_returns = pd.Series(index=returns.index)
    
    # Calculate portfolio returns using the optimized weights
    for i in range(len(returns)):
        if i >= len(optimal_weights) or optimal_weights.iloc[i].isnull().any():
            continue
        portfolio_returns.iloc[i] = np.sum(returns.iloc[i] * optimal_weights.iloc[i])
    
    return portfolio_returns

# Modified main function to include optimization
def main():
    # Tech Giants tickers
    tech_tickers = ['MSFT', 'AAPL', 'NVDA', 'TSLA', 'INTC']
    
    # Get data
    price_data = get_stock_data(tech_tickers, start_date, end_date)
    
    # Perform rolling optimization
    optimal_weights = rolling_optimization(price_data, window_size=60)
    
    # Calculate optimized portfolio performance
    optimized_returns = calculate_optimized_portfolio_performance(price_data, optimal_weights)
    
    # Calculate cumulative returns
    cumulative_returns = (1 + optimized_returns).cumprod()
    
    # Calculate metrics for optimized portfolio
    metrics = calculate_enhanced_metrics(optimized_returns)
    
    # Plot results
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Cumulative Returns
    plt.subplot(2, 1, 1)
    cumulative_returns.plot(label='Optimized Portfolio')
    plt.title('Cumulative Returns of Optimized Portfolio')
    plt.grid(True)
    plt.legend()
    
    # Plot 2: Rolling Weights
    plt.subplot(2, 1, 2)
    optimal_weights.plot.area(stacked=True)
    plt.title('Rolling Optimal Weights')
    plt.xlabel('Date')
    plt.ylabel('Weight')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    print("\nOptimized Portfolio Metrics:")
    metrics_df = pd.DataFrame(metrics, index=['Value']).T
    print(metrics_df)
    
    # Print average weights
    print("\nAverage Optimal Weights:")
    avg_weights = optimal_weights.mean()
    for ticker, weight in avg_weights.items():
        print(f"{ticker}: {weight:.2%}")
    
    return {
        'optimal_weights': optimal_weights,
        'portfolio_returns': optimized_returns,
        'cumulative_returns': cumulative_returns,
        'metrics': metrics
    }

if __name__ == "__main__":
    results = main()

NameError: name 'get_stock_data' is not defined